In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm

# read functions.
import os, random
for f in os.listdir('../fun/'): exec(open('../fun/'+f).read())
del f

# Load data
load( '../out/d3-fight-level-transform.pkl' )
print( X.shape )

random.seed( 1207 )

(4091, 161)


In [2]:
# train test split.
from sklearn.model_selection import train_test_split
X_train , X_test, y_train, y_test = train_test_split(
    X, y, 
    random_state = 744,
    test_size = 0.3
)

In [3]:
# run grid search on parameters.    
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV
m = AdaBoostClassifier()
readpriorgrid = False
if ( not readpriorgrid  )  or ( 'grid-adaboost.pkl' not in os.listdir('../out/') ):
    grid_adaboost = GridSearchCV(
        m, cv = 6, scoring = 'accuracy', n_jobs = -1, verbose = True,
        param_grid = dict(
            n_estimators = range( 40, 80, 5 ),
            learning_rate  = [ x/100 for x in range( 80, 120, 5 ) ],
            algorithm = ['SAMME', 'SAMME.R']
        )
    )
    grid_adaboost.fit( X_train, y_train )
    save( '../out/grid-adaboost.pkl', grid_adaboost )
else:
    load( '../out/grid-adaboost.pkl' )
print(grid_adaboost.best_params_)
print(grid_adaboost.best_score_)
m = grid_adaboost.best_estimator_

Fitting 6 folds for each of 128 candidates, totalling 768 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   14.3s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 768 out of 768 | elapsed:  4.2min finished


{'algorithm': 'SAMME', 'learning_rate': 1.0, 'n_estimators': 70}
0.6919315403422983


In [4]:
# Fit model and check in-model fit.
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
m.fit( X_train, y_train )
p = m.predict(X_train)
print({ 
    'fscore': f1_score( y_train, p ), 
    'precision': precision_score( y_train, p ), 
    'recall': recall_score( y_train, p ),
    'accuracy': accuracy_score( y_train, p )
})

{'fscore': 0.8006740491092923, 'precision': 0.7351900972590628, 'recall': 0.8789640591966174, 'accuracy': 0.7107928746070555}


In [5]:
# accuracy against test data.
p = m.predict(X_test)
print( { 
    'fscore': f1_score( y_test, p ), 
    'precision': precision_score( y_test, p ), 
    'recall': recall_score( y_test, p ),
    'accuracy': accuracy_score( y_test, p )
} )

{'fscore': 0.7626459143968871, 'precision': 0.694331983805668, 'recall': 0.8458692971639951, 'accuracy': 0.6522801302931596}


In [6]:
from sklearn.model_selection import cross_validate
kfold = cross_validate( 
    m, X, y, 
    cv = 10, 
    scoring = [ 'precision', 'recall', 'accuracy' ], 
    n_jobs = -1 
)
print( 
  'Mean precision: %s \nMean recall %s \n Mean Accuracy %s' % 
  ( kfold['test_precision'].mean(), kfold['test_recall'].mean(), kfold['test_accuracy'].mean() ) 
)

Mean precision: 0.7027541492074958 
Mean recall 0.8502583025830258 
 Mean Accuracy 0.6624603610303326


In [7]:
fi = pd.DataFrame({
    'feature': cols,
    'importance': m.feature_importances_
}).sort_values( 'importance', ascending = False )

fi[fi.importance>0].to_csv('ada-feature-import.csv',index=False)

print( fi[fi.importance>0] )

                            feature  importance
107                       Mean_odds    0.142725
108                       Diff_odds    0.130508
2                      no_of_rounds    0.098057
122          Diff_total_title_bouts    0.092010
85                 Mean_avg_SUB_ATT    0.079342
20            Referee_Kevin Mulhall    0.063461
87             Mean_avg_opp_SUB_ATT    0.056936
127  Mean_win_by_Decision_Unanimous    0.046568
144           Diff_avg_opp_BODY_pct    0.045157
143           Mean_avg_opp_BODY_pct    0.038568
1                      weight_class    0.030650
78                   Diff_Reach_cms    0.030441
115                     Mean_losses    0.027475
90                Diff_avg_BODY_pct    0.025816
82                    Diff_avg_PASS    0.022178
114         Diff_longest_win_streak    0.020467
126      Diff_win_by_Decision_Split    0.015868
32         Referee_Steve Mazzagatti    0.015573
140               Diff_avg_opp_PASS    0.011392
86                 Diff_avg_SUB_ATT    0

f1-score in-model is higher than on test indicating **over-fitting**. 

Test-set precision is still not great at 69% (predicting all-wins give 60%).